In [1]:
import librosa
import librosa.display
import numpy as np
import os
import statistics 
from tqdm.notebook import tqdm

In [2]:
l = os.listdir('Train')

In [3]:
lis=[]
for arch in l:
    if arch.endswith('.txt'):
        lis.append(arch)

In [4]:
sr=44100

In [5]:
esc = dict()
esc['A'] = 9
esc['B'] = 11
esc['C'] = 0
esc['D'] = 2
esc['E'] = 4
esc['F'] = 5
esc['G'] = 7
alt = dict()
alt['b'] = -1
alt['s'] = 1

In [6]:
ref = [32.7032, 34.6479, 36.7081, 38.8909, 
       41.2035, 43.6536, 46.2493, 24.4997, 
       25.9565, 27.5000, 29.1353, 30.8677]

In [7]:
aux=[]
for i in range(len(lis)):
    aux.append(lis[i].replace('.txt',''))
    aux[i] = aux[i].replace('_',' ')
    aux[i] = aux[i].split()

In [16]:
des

[2,
 2,
 2,
 2,
 -5,
 -2,
 3,
 4,
 -2,
 -5,
 -5,
 4,
 5,
 6,
 3,
 -4,
 -4,
 -4,
 0,
 0,
 3,
 3,
 3,
 2,
 1,
 1,
 1,
 0,
 0,
 3,
 0,
 0,
 3,
 3,
 3,
 4,
 5,
 1,
 1,
 1,
 0,
 0,
 -5,
 -5,
 -5,
 -5,
 0,
 -5,
 0,
 0,
 4,
 -1,
 -1,
 -1,
 6,
 6,
 6,
 6,
 1,
 1,
 3,
 3,
 0,
 0,
 3,
 -2,
 5,
 0,
 -5,
 -5,
 -5,
 2,
 2,
 -3,
 -3,
 -3,
 4,
 0,
 6,
 3,
 -4,
 5,
 4,
 1,
 1,
 2,
 2,
 1,
 4,
 0,
 3,
 1,
 1,
 1,
 0,
 -3,
 3,
 2,
 2,
 -5,
 -3,
 -3,
 -5,
 -5,
 -5,
 6,
 6,
 4,
 0,
 0,
 5,
 5,
 5,
 -5,
 -2,
 3,
 3,
 -1,
 4,
 4,
 0,
 0,
 0,
 5,
 -2,
 -2,
 4,
 3,
 3,
 4,
 0,
 0,
 4,
 4,
 1,
 1,
 4,
 -2,
 -1,
 -1,
 -1,
 5,
 2,
 0,
 -3,
 -3,
 -3,
 -3,
 -3,
 -3,
 5,
 -2,
 -2,
 3,
 3,
 -2,
 0,
 -2,
 3,
 3,
 0,
 -2,
 -2,
 -2,
 3,
 3,
 6,
 0,
 -4,
 -4,
 -4,
 0,
 5,
 0,
 -5,
 -5,
 -5,
 2,
 2,
 2,
 5,
 5,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 -2,
 3,
 -4,
 2,
 2,
 -3,
 3,
 3,
 -2,
 -2,
 -5,
 -5,
 -5,
 4,
 5,
 -5,
 -5,
 0,
 0,
 5,
 0]

In [8]:
des = []
for i in range(len(aux)):
    k = esc[aux[i][-2][0]]
    if len(aux[i][-2])>1:
        k = k + alt[aux[i][-2][1]]
    if 'm' in aux[i][-1]:
        k = k + 3
    k == k%12
    if k > 6:
        k = k - 12
    des.append(k)

In [9]:
for mus in tqdm(range(len(lis))):
    #Se carga el achivo de audio
    aux,sr = librosa.core.load('Train/'+lis[mus].replace('.txt','.wav'),sr=sr, mono=True)
    X = np.abs(librosa.cqt(aux, sr=sr, hop_length=512, n_bins=252, bins_per_octave=36, fmin=ref[des[mus]])).transpose()
    #Se lee el txt
    f=open('Train/'+lis[mus],'r')
    L=f.readlines()
    f.close()
    #Se guarda el inicio de la melodia
    inicio=librosa.time_to_frames(float(L[1].split()[0]), sr=sr)
    #Se recorta la melodia
    X = X[inicio:,:]
    #Se guarda el tamaño de la melodia
    nf = X.shape[0]
    #Se genera la matriz donde se guardara el piano roll
    y = np.zeros((nf, 88))
    #Se va guardando las notas
    for i in range(1,len(L)):
        k, j, p = float(L[i].split()[0]), float(L[i].split()[1]), int(L[i].split()[2])
        k, j, p = librosa.time_to_frames(k, sr=sr), librosa.time_to_frames(j, sr=sr), p-21-des[mus]
        y[(k-inicio):(j-inicio),p] = 1
    np.save('Train np 2/x_'+lis[mus].replace('.txt','')+'.npy', X)
    np.save('Train np 2/y_'+lis[mus].replace('.txt','')+'.npy', y)

  0%|          | 0/210 [00:00<?, ?it/s]

In [10]:
l = os.listdir('Test')

In [11]:
lis=[]
for arch in l:
    if arch.endswith('.txt'):
        lis.append(arch)

In [12]:
aux=[]
for i in range(len(lis)):
    aux.append(lis[i].replace('.txt',''))
    aux[i] = aux[i].replace('_',' ')
    aux[i] = aux[i].split()
des = []
for i in range(len(aux)):
    k = esc[aux[i][-2][0]]
    if len(aux[i][-2])>1:
        k = k + alt[aux[i][-2][1]]
    if 'm' in aux[i][-1]:
        k = k + 3
    k == k%12
    if k > 6:
        k = k - 12
    des.append(k)

In [13]:
for mus in tqdm(range(len(lis))):
    #Se carga el achivo de audio
    aux,sr = librosa.core.load('Test/'+lis[mus].replace('.txt','.wav'),sr=sr, mono=True)
    X = np.abs(librosa.cqt(aux, sr=sr, hop_length=512, n_bins=252, bins_per_octave=36, fmin=ref[des[mus]])).transpose()
    #Se lee el txt
    f=open('Test/'+lis[mus],'r')
    L=f.readlines()
    f.close()
    #Se guarda el inicio de la melodia
    inicio=librosa.time_to_frames(float(L[1].split()[0]), sr=sr)
    #Se recorta la melodia
    X = X[inicio:,:]
    #Se guarda el tamaño de la melodia
    nf = X.shape[0]
    #Se genera la matriz donde se guardara el piano roll
    y = np.zeros((nf, 88))
    #Se va guardando las notas
    for i in range(1,len(L)):
        k, j, p = float(L[i].split()[0]), float(L[i].split()[1]), int(L[i].split()[2])
        k, j, p = librosa.time_to_frames(k, sr=sr), librosa.time_to_frames(j, sr=sr), p-21-des[mus]
        y[(k-inicio):(j-inicio),p] = 1
    np.save('Test np 2/x_'+lis[mus].replace('.txt','')+'.npy', X)
    np.save('Test np 2/y_'+lis[mus].replace('.txt','')+'.npy', y)

  0%|          | 0/60 [00:00<?, ?it/s]